PyCitySchools Challenge Analysis

In [2]:
# Dependencies and Setup
from pathlib import Path
import pandas as pd

# File to Load
school_data_to_load = Path("Resources/schools_complete.csv")
student_data_to_load = Path("Resources/students_complete.csv")

# Read School and Student Data File and store into Pandas DataFrames
school_data_df = pd.read_csv(school_data_to_load)
student_data_df = pd.read_csv(student_data_to_load)

#Combine the data into a single dataset
school_data_complete_df = pd.merge(student_data_df, school_data_df, how="left", on=["school_name", "school_name"])

#school_data_complete_df.to_excel("school_data_complete.xlsx", index=False)
#I ran the above line of code myself so I can reference the full merged dataset separately

school_data_complete_df.head()


,Student ID,student_name,gender,year,school_name,reading_score,maths_score,School ID,type,size,budget
0,0,Paul Bradley,M,9,Huang High School,96,94,0,Government,2917,1910635
1,1,Victor Smith,M,12,Huang High School,90,43,0,Government,2917,1910635
2,2,Kevin Rodriguez,M,12,Huang High School,41,76,0,Government,2917,1910635
3,3,Richard Scott,M,12,Huang High School,89,86,0,Government,2917,1910635
4,4,Bonnie Ray,F,9,Huang High School,87,69,0,Government,2917,1910635


Local Government Area Summary

In [26]:
# Calculate the Totals (Schools and Students)
school_count = school_data_df["school_name"].count()
student_count = student_data_df["student_name"].count()

# Calculate the Total Budget
total_budget = school_data_df["budget"].sum()

school_count, student_count, total_budget

(15, 39170, 24649428)

In [27]:
# Calculate the Average Scores
average_maths_score = school_data_complete_df["maths_score"].mean()
average_reading_score = school_data_complete_df["reading_score"].mean()

average_maths_score, average_reading_score

(70.33819249425581, 69.98013786060761)

In [28]:
# Calculate the Percentage Pass Rates
passing_maths_count = school_data_complete_df[(school_data_complete_df["maths_score"] >= 50)].count()["student_name"]
passing_maths_percentage = passing_maths_count / float(student_count) * 100
passing_reading_count = school_data_complete_df[(school_data_complete_df["reading_score"] >= 50)].count()["student_name"]
passing_reading_percentage = passing_reading_count / float(student_count) * 100

passing_maths_reading_count = school_data_complete_df[(school_data_complete_df["maths_score"] >= 50) & (school_data_complete_df["reading_score"] >= 50)].count()["student_name"]

overall_passing_rate = passing_maths_reading_count / float(student_count) * 100

passing_maths_count, passing_maths_percentage, passing_reading_count, passing_reading_percentage, passing_maths_reading_count, overall_passing_rate

(33717, 86.07863160582077, 33070, 84.42685728874139, 28519, 72.80827163645647)

In [29]:
# Convert to DataFrame
headings = ["Total Schools", "Total Students", "Total Budget", "Average Maths Score", "Average Reading Score", 
            "% Passing Maths", "% Passing Reading", "% Overall Passing"]
values = [[school_count, student_count, total_budget, average_maths_score, average_reading_score, 
           passing_maths_percentage, passing_reading_percentage, overall_passing_rate]]
area_summary = pd.DataFrame(values, columns=headings)

# Formatting
area_summary["Total Students"] = area_summary["Total Students"].map("{:,}".format)
area_summary["Total Budget"] = area_summary["Total Budget"].map("${:,.2f}".format)

# Display the DataFrame
area_summary.head()

,Total Schools,Total Students,Total Budget,Average Maths Score,Average Reading Score,% Passing Maths,% Passing Reading,% Overall Passing
0,15,"39,170","$24,649,428.00",70.338192,69.980138,86.078632,84.426857,72.808272


School Summary

In [30]:
# Select the type per school from school_data
school_types = school_data_complete_df.groupby("school_name")["type"].first()

# Calculate the total student count per school from school_data
per_school_counts = school_data_complete_df.groupby("school_name")["size"].count()

# Calculate the total school budget and per capita (per student) spending per school from school_data
per_school_budget = school_data_complete_df.groupby("school_name")["budget"].sum() / per_school_counts
per_school_capita = per_school_budget / per_school_counts

# Calculate the average test scores per school from school_data_complete
per_school_maths = school_data_complete_df.groupby("school_name")["maths_score"].mean()
per_school_reading = school_data_complete_df.groupby("school_name")["reading_score"].mean()

#school_types, per_school_counts, per_school_budget, per_school_capita, per_school_maths, per_school_reading

In [31]:
# Get the students who passed maths and passed reading by creating separate filtered DataFrames from school_data_complete.

school_passing_maths = school_data_complete_df[school_data_complete_df["maths_score"] >= 50].groupby("school_name").size()
school_passing_reading = school_data_complete_df[school_data_complete_df["reading_score"] >= 50].groupby("school_name").size()

# Get the students who passed both reading and maths in a separate DataFrame from school_data_complete.
passing_maths_and_reading = school_data_complete_df[
    (school_data_complete_df["maths_score"] >= 50) & 
    (school_data_complete_df["reading_score"] >= 50)
].groupby("school_name").size()

In [32]:
#  Calculate the Percentage Pass Rates
per_school_passing_maths = school_passing_maths / per_school_counts * 100
per_school_passing_reading = school_passing_reading / per_school_counts * 100
overall_passing_rate = passing_maths_and_reading / per_school_counts * 100

In [34]:
# Convert to DataFrame
per_school_summary = pd.DataFrame({
    "School Type": school_types,
    "Total Students": per_school_counts,
    "Total School Budget": per_school_budget,
    "Per Student Budget": per_school_capita,
    "Average Maths Score": per_school_maths,
    "Average Reading Score": per_school_reading,
    "% Passing Maths": per_school_passing_maths,
    "% Passing Reading": per_school_passing_reading,
    "% Overall Passing": overall_passing_rate})

# Formatting
per_school_summary["Total School Budget"] = per_school_summary["Total School Budget"].map("${:,.2f}".format)
per_school_summary["Per Student Budget"] = per_school_summary["Per Student Budget"].map("${:,.2f}".format)

per_school_summary = per_school_summary.reset_index()

# Display the DataFrame
per_school_summary

,school_name,School Type,Total Students,Total School Budget,Per Student Budget,Average Maths Score,Average Reading Score,% Passing Maths,% Passing Reading,% Overall Passing
0,Bailey High School,Government,4976,"$3,124,928.00",$628.00,72.352894,71.008842,91.639871,87.379421,80.084405
1,Cabrera High School,Independent,1858,"$1,081,356.00",$582.00,71.657158,71.359526,90.850377,89.074273,80.785791
2,Figueroa High School,Government,2949,"$1,884,411.00",$639.00,68.698542,69.077993,81.654798,82.807731,67.650051
3,Ford High School,Government,2739,"$1,763,916.00",$644.00,69.091274,69.572472,82.438846,82.219788,67.469880
4,Griffin High School,Independent,1468,"$917,500.00",$625.00,71.788147,71.245232,91.212534,88.487738,81.335150
5,Hernandez High School,Government,4635,"$3,022,020.00",$652.00,68.874865,69.186408,80.949299,81.877023,66.364617
6,Holden High School,Independent,427,"$248,087.00",$581.00,72.583138,71.660422,89.929742,88.524590,78.922717
7,Huang High School,Government,2917,"$1,910,635.00",$655.00,68.935207,68.910525,81.693521,81.453548,66.712376
8,Johnson High School,Government,4761,"$3,094,650.00",$650.00,68.843100,69.039277,82.062592,81.978576,67.191766
9,Pena High School,Independent,962,"$585,858.00",$609.00,72.088358,71.613306,91.683992,86.590437,79.209979


Highest-Performing Schools (by % Overall Passing)

In [35]:
# Sort by descending order and show top five schools
top_schools = per_school_summary.sort_values(by="% Overall Passing", ascending=False)
top_schools.head(5)

,school_name,School Type,Total Students,Total School Budget,Per Student Budget,Average Maths Score,Average Reading Score,% Passing Maths,% Passing Reading,% Overall Passing
4,Griffin High School,Independent,1468,"$917,500.00",$625.00,71.788147,71.245232,91.212534,88.487738,81.335150
1,Cabrera High School,Independent,1858,"$1,081,356.00",$582.00,71.657158,71.359526,90.850377,89.074273,80.785791
0,Bailey High School,Government,4976,"$3,124,928.00",$628.00,72.352894,71.008842,91.639871,87.379421,80.084405
14,Wright High School,Independent,1800,"$1,049,400.00",$583.00,72.047222,70.969444,91.777778,86.666667,79.722222
10,Rodriguez High School,Government,3999,"$2,547,363.00",$637.00,72.047762,70.935984,90.797699,87.396849,79.419855


Lowest-Performing Schools (by % Overall Passing)

In [36]:
# Sort by ascending order and show bottom five schools
bottom_schools = per_school_summary.sort_values(by="% Overall Passing", ascending=True)
bottom_schools.head(5)

,school_name,School Type,Total Students,Total School Budget,Per Student Budget,Average Maths Score,Average Reading Score,% Passing Maths,% Passing Reading,% Overall Passing
5,Hernandez High School,Government,4635,"$3,022,020.00",$652.00,68.874865,69.186408,80.949299,81.877023,66.364617
7,Huang High School,Government,2917,"$1,910,635.00",$655.00,68.935207,68.910525,81.693521,81.453548,66.712376
8,Johnson High School,Government,4761,"$3,094,650.00",$650.00,68.843100,69.039277,82.062592,81.978576,67.191766
13,Wilson High School,Independent,2283,"$1,319,574.00",$578.00,69.170828,68.876916,82.785808,81.296540,67.455103
3,Ford High School,Government,2739,"$1,763,916.00",$644.00,69.091274,69.572472,82.438846,82.219788,67.469880


Maths Scores by Year

In [38]:
# Create data series of scores by year levels using conditionals
year_nine = school_data_complete_df[(school_data_complete_df["year"] == 9)]
year_ten = school_data_complete_df[(school_data_complete_df["year"] == 10)]
year_eleven = school_data_complete_df[(school_data_complete_df["year"] == 11)]
year_twelve = school_data_complete_df[(school_data_complete_df["year"] == 12)]

# Group each by school name
year_nine_scores = year_nine.groupby("school_name")["maths_score"].mean().rename("Year 9")
year_ten_scores = year_ten.groupby("school_name")["maths_score"].mean().rename("Year 10")
year_eleven_scores = year_eleven.groupby("school_name")["maths_score"].mean().rename("Year 11")
year_twelve_scores =year_twelve.groupby("school_name")["maths_score"].mean().rename("Year 12")

#Combine series into single DataFrame
maths_scores_by_year = pd.concat([year_nine_scores, year_ten_scores, year_eleven_scores, year_twelve_scores], axis=1)

#Minor data wrangling
maths_scores_by_year.index.name = None

#Display the DataFrame
maths_scores_by_year

,Year 9,Year 10,Year 11,Year 12
Bailey High School,72.493827,71.897498,72.374900,72.675097
Cabrera High School,72.321970,72.437768,71.008299,70.604712
Figueroa High School,68.477804,68.331586,68.811001,69.325282
Ford High School,69.021609,69.387006,69.248862,68.617811
Griffin High School,72.789731,71.093596,71.692521,71.469178
Hernandez High School,68.586831,68.867156,69.154412,68.985075
Holden High School,70.543307,75.105263,71.640777,73.409639
Huang High School,69.081754,68.533246,69.431345,68.639316
Johnson High School,69.469286,67.990220,68.637730,69.287393
Pena High School,71.996364,72.396000,72.523438,71.187845


Reading Scores by Year

In [39]:
# Create data series of scores by year levels using conditionals
year_nine = school_data_complete_df[(school_data_complete_df["year"] == 9)]
year_ten = school_data_complete_df[(school_data_complete_df["year"] == 10)]
year_eleven = school_data_complete_df[(school_data_complete_df["year"] == 11)]
year_twelve = school_data_complete_df[(school_data_complete_df["year"] == 12)]


# Group each by school name
year_nine_scores = year_nine.groupby("school_name")["reading_score"].mean().rename("Year 9")
year_ten_scores = year_ten.groupby("school_name")["reading_score"].mean().rename("Year 10")
year_eleven_scores = year_eleven.groupby("school_name")["reading_score"].mean().rename("Year 11")
year_twelve_scores =year_twelve.groupby("school_name")["reading_score"].mean().rename("Year 12")

 
# Combine series into single DataFrame
reading_scores_by_year = pd.concat([year_nine_scores, year_ten_scores, year_eleven_scores, year_twelve_scores], axis=1)

# Minor data wrangling
reading_scores_by_year.index.name = None

# Display the DataFrame
reading_scores_by_year

,Year 9,Year 10,Year 11,Year 12
Bailey High School,70.901920,70.848265,70.317346,72.195525
Cabrera High School,71.172348,71.328326,71.201245,71.856021
Figueroa High School,70.261682,67.677588,69.152327,69.082126
Ford High School,69.615846,68.988701,70.735964,68.849722
Griffin High School,72.026895,70.746305,72.385042,69.434932
Hernandez High School,68.477569,70.621842,68.418199,69.244136
Holden High School,71.598425,71.096491,73.310680,70.481928
Huang High School,68.670616,69.516297,68.740638,68.671795
Johnson High School,68.719286,69.295029,69.969115,67.992521
Pena High School,70.949091,72.324000,71.703125,71.513812


Scores by School Spending

In [40]:
# Establish the bins
spending_bins = [0, 585, 630, 645, 680]
group_names = ["<$585", "$585-630", "$630-645", "$645-680"]

In [41]:
# Create a copy of the school summary since it has the "Per Student Budget"
#  This step can be skipped but its best to make a copy.
school_spending_df = per_school_summary
school_spending_df.head()


,school_name,School Type,Total Students,Total School Budget,Per Student Budget,Average Maths Score,Average Reading Score,% Passing Maths,% Passing Reading,% Overall Passing
0,Bailey High School,Government,4976,"$3,124,928.00",$628.00,72.352894,71.008842,91.639871,87.379421,80.084405
1,Cabrera High School,Independent,1858,"$1,081,356.00",$582.00,71.657158,71.359526,90.850377,89.074273,80.785791
2,Figueroa High School,Government,2949,"$1,884,411.00",$639.00,68.698542,69.077993,81.654798,82.807731,67.650051
3,Ford High School,Government,2739,"$1,763,916.00",$644.00,69.091274,69.572472,82.438846,82.219788,67.469880
4,Griffin High School,Independent,1468,"$917,500.00",$625.00,71.788147,71.245232,91.212534,88.487738,81.335150


In [43]:
# Categorise spending based on the bins.
# Use pd.cut to categorise spending based on the bins

# Revert Per Student Budget format back to numeric
school_spending_df['Per Student Budget'] = school_spending_df['Per Student Budget'].replace('[\$,]', '', regex=True).astype(float)
#school_spending_df.head()

school_spending_df["Spending Ranges (Per Student)"] = pd.cut(school_spending_df["Per Student Budget"], spending_bins, labels=group_names, include_lowest=True)

# Reformat Per Student Budget back to previous
per_school_summary["Per Student Budget"] = per_school_summary["Per Student Budget"].map("${:,.2f}".format)

school_spending_df

,school_name,School Type,Total Students,Total School Budget,Per Student Budget,Average Maths Score,Average Reading Score,% Passing Maths,% Passing Reading,% Overall Passing,Spending Ranges (Per Student)
0,Bailey High School,Government,4976,"$3,124,928.00",$628.00,72.352894,71.008842,91.639871,87.379421,80.084405,$585-630
1,Cabrera High School,Independent,1858,"$1,081,356.00",$582.00,71.657158,71.359526,90.850377,89.074273,80.785791,<$585
2,Figueroa High School,Government,2949,"$1,884,411.00",$639.00,68.698542,69.077993,81.654798,82.807731,67.650051,$630-645
3,Ford High School,Government,2739,"$1,763,916.00",$644.00,69.091274,69.572472,82.438846,82.219788,67.469880,$630-645
4,Griffin High School,Independent,1468,"$917,500.00",$625.00,71.788147,71.245232,91.212534,88.487738,81.335150,$585-630
5,Hernandez High School,Government,4635,"$3,022,020.00",$652.00,68.874865,69.186408,80.949299,81.877023,66.364617,$645-680
6,Holden High School,Independent,427,"$248,087.00",$581.00,72.583138,71.660422,89.929742,88.524590,78.922717,<$585
7,Huang High School,Government,2917,"$1,910,635.00",$655.00,68.935207,68.910525,81.693521,81.453548,66.712376,$645-680
8,Johnson High School,Government,4761,"$3,094,650.00",$650.00,68.843100,69.039277,82.062592,81.978576,67.191766,$645-680
9,Pena High School,Independent,962,"$585,858.00",$609.00,72.088358,71.613306,91.683992,86.590437,79.209979,$585-630


In [44]:
#  Calculate averages for the desired columns.
spending_maths_scores = school_spending_df.groupby(["Spending Ranges (Per Student)"])["Average Maths Score"].mean()
spending_reading_scores = school_spending_df.groupby(["Spending Ranges (Per Student)"])["Average Reading Score"].mean()
spending_passing_maths = school_spending_df.groupby(["Spending Ranges (Per Student)"])["% Passing Maths"].mean()
spending_passing_reading = school_spending_df.groupby(["Spending Ranges (Per Student)"])["% Passing Reading"].mean()
overall_passing_spending = school_spending_df.groupby(["Spending Ranges (Per Student)"])["% Overall Passing"].mean()

C:\Users\kenne\AppData\Local\Temp\ipykernel_1776\1079946528.py:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  spending_maths_scores = school_spending_df.groupby(["Spending Ranges (Per Student)"])["Average Maths Score"].mean()
C:\Users\kenne\AppData\Local\Temp\ipykernel_1776\1079946528.py:3: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  spending_reading_scores = school_spending_df.groupby(["Spending Ranges (Per Student)"])["Average Reading Score"].mean()
C:\Users\kenne\AppData\Local\Temp\ipykernel_1776\1079946528.py:4: FutureWarning: The default of observed=False is deprecated and will be changed to True in 

In [46]:
# Assemble into DataFrame
spending_summary = pd.concat([
    spending_maths_scores,
    spending_reading_scores,
    spending_passing_maths,
    spending_passing_reading,
    overall_passing_spending
], axis=1)

# Rename columns
spending_summary.columns = ["Average Maths Score", "Average Reading Score", "% Passing Maths", "% Passing Reading", "% Overall Passing"]

# Display results
spending_summary

,Average Maths Score,Average Reading Score,% Passing Maths,% Passing Reading,% Overall Passing
Spending Ranges (Per Student),,,,,
<$585,71.364587,70.716577,88.835926,86.390517,76.721458
$585-630,72.065868,71.031297,91.518824,87.292423,79.876293
$630-645,69.854807,69.838814,84.686139,83.763585,71.004977
$645-680,68.884391,69.045403,81.568470,81.769716,66.756253


Scores by School Size

In [47]:
# Establish the bins.
size_bins = [0, 1000, 2000, 5000]
group_names = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]


In [48]:
# Categorize the spending based on the bins
per_school_summary["School Size"] = pd.cut(per_school_summary["Total Students"], size_bins, labels=group_names, include_lowest=True)
per_school_summary

,school_name,School Type,Total Students,Total School Budget,Per Student Budget,Average Maths Score,Average Reading Score,% Passing Maths,% Passing Reading,% Overall Passing,Spending Ranges (Per Student),School Size
0,Bailey High School,Government,4976,"$3,124,928.00",$628.00,72.352894,71.008842,91.639871,87.379421,80.084405,$585-630,Large (2000-5000)
1,Cabrera High School,Independent,1858,"$1,081,356.00",$582.00,71.657158,71.359526,90.850377,89.074273,80.785791,<$585,Medium (1000-2000)
2,Figueroa High School,Government,2949,"$1,884,411.00",$639.00,68.698542,69.077993,81.654798,82.807731,67.650051,$630-645,Large (2000-5000)
3,Ford High School,Government,2739,"$1,763,916.00",$644.00,69.091274,69.572472,82.438846,82.219788,67.469880,$630-645,Large (2000-5000)
4,Griffin High School,Independent,1468,"$917,500.00",$625.00,71.788147,71.245232,91.212534,88.487738,81.335150,$585-630,Medium (1000-2000)
5,Hernandez High School,Government,4635,"$3,022,020.00",$652.00,68.874865,69.186408,80.949299,81.877023,66.364617,$645-680,Large (2000-5000)
6,Holden High School,Independent,427,"$248,087.00",$581.00,72.583138,71.660422,89.929742,88.524590,78.922717,<$585,Small (<1000)
7,Huang High School,Government,2917,"$1,910,635.00",$655.00,68.935207,68.910525,81.693521,81.453548,66.712376,$645-680,Large (2000-5000)
8,Johnson High School,Government,4761,"$3,094,650.00",$650.00,68.843100,69.039277,82.062592,81.978576,67.191766,$645-680,Large (2000-5000)
9,Pena High School,Independent,962,"$585,858.00",$609.00,72.088358,71.613306,91.683992,86.590437,79.209979,$585-630,Small (<1000)


In [49]:
# Calculate averages for the desired columns.
size_maths_scores = per_school_summary.groupby(["School Size"])["Average Maths Score"].mean()
size_reading_scores = per_school_summary.groupby(["School Size"])["Average Reading Score"].mean()
size_passing_maths = per_school_summary.groupby(["School Size"])["% Passing Maths"].mean()
size_passing_reading = per_school_summary.groupby(["School Size"])["% Passing Reading"].mean()
size_overall_passing = per_school_summary.groupby(["School Size"])["% Overall Passing"].mean()

C:\Users\kenne\AppData\Local\Temp\ipykernel_1776\2220874659.py:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  size_maths_scores = per_school_summary.groupby(["School Size"])["Average Maths Score"].mean()
C:\Users\kenne\AppData\Local\Temp\ipykernel_1776\2220874659.py:3: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  size_reading_scores = per_school_summary.groupby(["School Size"])["Average Reading Score"].mean()
C:\Users\kenne\AppData\Local\Temp\ipykernel_1776\2220874659.py:4: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=Fa

In [50]:
# Assemble into DataFrame
size_summary = pd.concat([
    size_maths_scores,
    size_reading_scores,
    size_passing_maths,
    size_passing_reading,
    size_overall_passing
], axis=1)

# Rename columns
spending_summary.columns = ["Average Maths Score", "Average Reading Score", "% Passing Maths", "% Passing Reading", "% Overall Passing"]

# Display results
size_summary

,Average Maths Score,Average Reading Score,% Passing Maths,% Passing Reading,% Overall Passing
School Size,,,,,
Small (<1000),72.335748,71.636864,90.806867,87.557513,79.066348
Medium (1000-2000),71.421650,70.720164,89.846560,86.714149,78.039785
Large (2000-5000),69.751809,69.576052,84.252804,83.301185,70.293507


Scores by School Type

In [51]:
# Create new series using groupby for:
# Type | Average Maths Score | Average Reading Score | % Passing Maths | % Passing Reading | % Overall Passing

type_maths_scores = per_school_summary.groupby(["School Type"])["Average Maths Score"].mean()
type_reading_scores = per_school_summary.groupby(["School Type"])["Average Reading Score"].mean()
type_passing_maths = per_school_summary.groupby(["School Type"])["% Passing Maths"].mean()
type_passing_reading = per_school_summary.groupby(["School Type"])["% Passing Reading"].mean()
type_overall_passing = per_school_summary.groupby(["School Type"])["% Overall Passing"].mean()

In [52]:
# Assemble into DataFrame
type_summary = pd.concat([
    type_maths_scores,
    type_reading_scores,
    type_passing_maths,
    type_passing_reading,
    type_overall_passing
], axis=1)

# Rename columns
spending_summary.columns = ["Average Maths Score", "Average Reading Score", "% Passing Maths", "% Passing Reading", "% Overall Passing"]

# Display results
type_summary

,Average Maths Score,Average Reading Score,% Passing Maths,% Passing Reading,% Overall Passing
School Type,,,,,
Government,69.834806,69.675929,84.462375,83.587562,70.698993
Independent,71.368822,70.718933,89.204043,86.247789,76.973340
